# Vison 
Here are the core codes and respective explanation of the vison information processing module.

## Obtaining the central coordinates of the target object

Image acquisition and preprocessing: Here, RGB and depth images are obtained through ROS, and the format is converted using CvBridge. Camera intrinsic parameters are also retrieved to facilitate subsequent coordinate transformations.

In [ ]:
bridge = CvBridge()
cv_image = bridge.imgmsg_to_cv2(img, "bgr8")
depth_image = bridge.imgmsg_to_cv2(depth_img, "passthrough")
cv2.imwrite("save.jpg", cv_image)

fx, fy, cx, cy = camera_info.K[0], camera_info.K[4],
camera_info.K[2], camera_info.K[5]
camera_matrix = np.array([[fx, 0, cx], [0, fy, cy], [0, 0, 1]])

Define the color range and apply color filtering: Here, the color range of the target object is defined, and the RGB image is converted to the HSV color space. A mask image is obtained by applying the color filter.

In [ ]:
color_ranges = {
  "green": ([50, 70, 80], [90, 255, 255]),
  "purple": ([75, 75, 115], [255, 145, 195]),
  "pink": ([150, 80, 100], [200, 200, 255]),
  "pink2": ([0, 130, 130], [30, 170, 170]),
}

hsv_frame = cv2.cvtColor(roi, cv2.COLOR_BGR2HSV)
mask_all = None
filtered_contours = []

Contour detection and center coordinate calculation: Contour detection is used to obtain the contours of the target object, and the vertices of the contour are obtained through polygon approximation. The contour with the largest area is selected, and its center coordinates are calculated.

In [ ]:
for color, (lower, upper) in color_ranges.items():
    lower = np.array(lower)
    upper = np.array(upper)
    mask = cv2.inRange(hsv_frame, lower, upper)
    num_labels, labels, stats, centroids =
cv2.connectedComponentsWithStats(mask)

    min_area_threshold = 100
    for i in range(1, num_labels):
        area = stats[i, cv2.CC_STAT_AREA]
        if area < min_area_threshold:
            labels[labels == i] = 0

    mask = np.where(labels > 0, 255, 0).astype(np.uint8)
    if mask_all is None:
        mask_all = mask
    else:
        mask_all = cv2.bitwise_or(mask_all, mask)
        
contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL,
cv2.CHAIN_APPROX_SIMPLE)

for contour in contours:
    approx = cv2.approxPolyDP(contour, 0.01 *
cv2.arcLength(contour, True), True)
    if len(approx) <= 8:
        filtered_contours.append((contour, color))

Calculation of the 3D coordinates of the grasping point: By using the pixel coordinates of the object’s center and combining them with depth image information, the 3D coordinates are calculated.

In [ ]:
M = cv2.moments(max_contour)

if M["m00"] != 0:
    center_x = int(M["m10"] / M["m00"]) + x1
    center_y = int(M["m01"] / M["m00"]) + y1
    center = (center_x, center_y)

else:
    center = None
    
u, v = center
center_3d = contour23D(u, v)
center_x, center_y, center_z = center_3d

## Estimation of the surface normal based on point cloud information

Reading point cloud data and constructing a KD tree: Point cloud data is retrieved from the point cloud message, and a point cloud object is constructed. Downsampling is performed, and a KD tree is built to quickly query neighboring points.


In [ ]:
points = np.array(list(pc2.read_points(point_cloud, skip_nans=True,
field_names=("x", "y", "z"))))
pc = o3d.geometry.PointCloud()
pc.points = o3d.utility.Vector3dVector(points)
pc = pc.voxel_down_sample(voxel_size=0.003)
pc_tree = o3d.geometry.KDTreeFlann(pc)

Obtaining point cloud data near the grasping point: Query the point cloud near the grasping point and create a new point cloud object for the neighboring points.

In [ ]:
center_3d = np.array([center_x, center_y, center_z])
radius = 0.03
[k, idx, _] = pc_tree.search_radius_vector_3d(center_3d, radius)
nearby_points = np.asarray(pc.points)[idx, :] - center_3d
nearby_pc = o3d.geometry.PointCloud()
nearby_pc.points = o3d.utility.Vector3dVector(nearby_points)

ICP alignment and surface normal calculation: Using the ICP alignment method, the neighborhood point cloud is aligned with the ideal plane, the rotation matrix is obtained, and the surface normal is calculated.

In [ ]:
target_points = [[center_3d[0], center_3d[1], center_3d[2]]]
num_random_pnts = 500
random_r = radius * np.sqrt(np.random.uniform(0, 1,
num_random_pnts))
random_theta = np.random.uniform(0, 2 * np.pi, num_random_pnts)
for i in range(num_random_pnts):
    target_points.append([random_r[i] * np.cos(random_theta[i]),
random_r[i] * np.sin(random_theta[i]), 0])

target_points = np.array(target_points)
target = o3d.geometry.PointCloud()
target.points = o3d.utility.Vector3dVector(target_points)

threshold = 0.02
trans_init = np.identity(4)

reg_p2p = o3d.pipelines.registration.registration_icp(
  nearby_pc, target, threshold, trans_init,
 o3d.pipelines.registration.TransformationEstimationPointToPoint(),
 o3d.pipelines.registration.ICPConvergenceCriteria(max_iteration=20
00) )

rotation_matrix = reg_p2p.transformation
ideal_normal = np.array([0, 0, 1])
normal1 = ideal_normal.dot((rotation_matrix[:3, :3]))

## Publishing color and results 
Based on the color of the block with the largest detected area, the color information is published.

The corresponding information is published to the tf node (including the surface normal and coordinates of the grasping point) and the ros topic (including the color of the grasped block).

In [ ]:
global color_pub
if max_color == "green":
    color_pub.publish("green")
elif max_color == "purple":
    color_pub.publish("purple")
elif max_color == "pink" or max_color == "pink2":
color_pub.publish("red")